In [1]:
import torch
import numpy as np
import wandb

from utils import parse_config, unflatten_dot
from dataset import get_loader
from learner import Learner
from models import ASTPretrained
from types import SimpleNamespace

In [2]:
SEED = 123
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

In [3]:
def main():
    
    wandb.init()
    config = wandb.config
    train_dl = get_loader(config, subset="train")
    valid_dl = get_loader(config, subset="valid")
    
    model = ASTPretrained(n_classes=11)

    learn = Learner(train_dl, valid_dl, model, config)
    learn.unfreeze()

    learn.fit(4)

In [4]:
import yaml
CONFIG_PATH = "./sweep_config.yaml"
with open(CONFIG_PATH) as file:
    config = yaml.safe_load(file)

sweep_id = wandb.sweep(sweep=config)
wandb.agent(sweep_id, function=main, count=1)

Create sweep with ID: nkjq1fa2
Sweep URL: https://wandb.ai/k-pintaric/uncategorized/sweeps/nkjq1fa2


wandb: Agent Starting Run: hjztcig1 with config:
wandb: 	EPOCHS: 8
wandb: 	LLRD: {'base_lr': 5.216780526316783e-05, 'lr_decay_rate': 0.7724874615846169, 'weight_decay': 7.20763066146665e-07}
wandb: 	batch_size: 8
wandb: 	loss: {'FocalLoss': {'alpha': 0.25, 'gamma': 3}}
wandb: 	metrics: ['hamming_score', 'zero_one_score', 'mAP', 'mean_f1_score']
wandb: 	num_accum: 6
wandb: 	optimizer: {'AdamW': {'lr': 2.5876328734075884e-05, 'weight_decay': 0}}
wandb: 	preprocess: {'PreprocessPipeline': {'target_sr': 16000}}
wandb: 	save_best_model: True
wandb: 	scheduler: {'CosineAnnealingLR': {'eta_min': 0}}
wandb: 	signal_augments: {'RepeatAudio': {'max_repeats': 4}}
wandb: 	spec_augments: {'MaskFrequency': {'max_mask_length': 30}, 'MaskTime': {'max_mask_length': 49}}
wandb: 	train_dir: ./data/processed/all_sync/IRMAS_Training_Data
wandb: 	transforms: {'FeatureExtractor': {'sr': 16000}}
wandb: 	valid_dir: ./data/raw/IRMAS_Validation_Data
wandb: 	verbose: True
wandb: Currently logged in as: k-pintaric

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/9220 [00:00<?, ?it/s]

Run hjztcig1 errored: AttributeError('Caught AttributeError in DataLoader worker process 0.\nOriginal Traceback (most recent call last):\n  File "/nfs/home/912c2e22633dde2e/lumen-irmas/venv/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop\n    data = fetcher.fetch(index)\n  File "/nfs/home/912c2e22633dde2e/lumen-irmas/venv/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch\n    data = [self.dataset[idx] for idx in possibly_batched_index]\n  File "/nfs/home/912c2e22633dde2e/lumen-irmas/venv/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>\n    data = [self.dataset[idx] for idx in possibly_batched_index]\n  File "/nfs/home/912c2e22633dde2e/lumen-irmas/dataset.py", line 52, in __getitem__\n    signal = self.transforms(signal)\n  File "/nfs/home/912c2e22633dde2e/lumen-irmas/transforms.py", line 106, in __call__\n    return self.transform(signal.squeeze()).input_values.squeeze()\nA